In [ ]:
from __future__ import print_function, division
import os, bz2, json, time, sys, csv
from datetime import timedelta, datetime

import math
import collections
import numpy as np
import pandas as pd
import matplotlib as mpl
import scipy.stats as stats

from matplotlib import pyplot as plt
from tqdm import tqdm

from util_read import engage_read, read_vids, engage_tweets_read, merge_engage_tweets, engage_metadata, compute_period_daily

In [ ]:
# file_vids_filter_all = "/home/users/u6314203/pyProject/hip-popularity/data/engage16/filtered/vids_filter_all.csv" 
# engage_dataset_base = "/localdata/u6314203/dataset_engage16/tweeted_videos" 
# tweets_dataset_base = "/localdata/u6314203/dataset_tweets_engage/dataset_tweets_fixTZ"

file_vids_filter_all = "C:/Users/u6314203/pyProjects/hip-popularity/data/engage16/filtered/vids_filter_all.csv"
engage_dataset_base = "G:/MLCV dataset/engagement'16/tweeted_videos"
tweets_dataset_base = "G:/MLCV dataset/engagement'16/tweets_dataset_fixTZ"

categories_eligible = ["autos", "comedy","education","entertainment", \
                       "film","gaming","howto","music",\
                       "news","people","science","sports",\
                       "travel"]

In [ ]:
# vids from all category
vids_filter_all = read_vids(file_vids_filter_all)

merged_dataset = dict()
# engage_metadata = dict() 
for category in categories_eligible[:]:
    engage_dataset = engage_read(engage_dataset_base, category)
    tweets_dataset = engage_tweets_read(tweets_dataset_base, category)
    
    curr_merged = merge_engage_tweets(engage_dataset, tweets_dataset)
    merged_dataset.update(curr_merged)
    
print(f"Total size: {len(merged_dataset)}")

In [ ]:
timepoint = 90
pearsonr_vals = []
vids = []
for k, vals in merged_dataset.items():
    series_views = vals["viewCounts"][timepoint:]
    series_tweets = vals["tweets"][timepoint:]
    r, p = stats.pearsonr(series_views, series_tweets)
    pearsonr_vals.append([r,p])
    vids.append(k)

df_pearsonr = pd.DataFrame(pearsonr_vals, columns=["r", "p"], index=vids)
df_pearsonr = df_pearsonr[[not x for x in np.isnan(df_pearsonr["r"])]]
pd.DataFrame.describe(df_pearsonr)

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(1,2,1)
n, bins, patches = ax.hist(df_pearsonr["r"], bins=20)
ax.set_title(f"Distribution of pearsonr results: comedy")
ax.set_xlabel("pearsonr r value")
ax.set_ylabel("Number of videos")

ax = fig.add_subplot(1,2,2)
ax.scatter(df_pearsonr["p"], df_pearsonr["r"])
# ax.set_title(f"Distribution of pearsonr results: comedy")
ax.set_xlabel("pearsonr p value")
ax.set_ylabel("pearsonr r value")

In [ ]:
df_filter = df_pearsonr[df_pearsonr["p"]<=0.05]

pd.DataFrame.describe(df_filter)

In [ ]:
fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(1,2,1)
n, bins, patches = ax.hist(df_filter["r"], bins=20)
ax.set_title(f"Distribution of pearsonr results(p<=0.05): comedy")
ax.set_xlabel("pearsonr r value")
ax.set_ylabel("Number of videos")

ax = fig.add_subplot(1,2,2)
ax.scatter(df_filter["p"], df_filter["r"])
# ax.set_title(f"Distribution of pearsonr results: comedy")
ax.set_xlabel("pearsonr p value")
ax.set_ylabel("pearsonr r value")